In [1]:
import pandas as pd # DATAFRAMES
import matplotlib.pyplot as plt # GRAPHIC LIBRARIES
import plotly.express as px # INTERACTIVE GRAPHICS & MORE COMPLEX

In [2]:
# INSTALLS DIRECTLY FROM GITHUB & SAVES IT AS AN XLSX
!wget -O inventory.xlsx https://github.com/jordipereiragude/dataforcourses/raw/refs/heads/main/inventory.xlsx

--2025-02-20 17:56:07--  https://github.com/jordipereiragude/dataforcourses/raw/refs/heads/main/inventory.xlsx
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jordipereiragude/dataforcourses/refs/heads/main/inventory.xlsx [following]
--2025-02-20 17:56:08--  https://raw.githubusercontent.com/jordipereiragude/dataforcourses/refs/heads/main/inventory.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3998305 (3.8M) [application/octet-stream]
Saving to: ‘inventory.xlsx’

inventory.xlsx      100%[===================>]   3.81M  --.-KB/s    in 0.02s   

2025-02-20 17:56:08 (169 MB/s) - ‘inventory.xl

In [3]:
df = pd.read_excel('inventory.xlsx')

In [4]:
#echemos un vistazo a los datos
print(df)
#veamos los tipos de datos
print(df.dtypes)

                   Order ID      Date              SKU  Qty  Amount
0       171-9198151-1101146  04-30-22  JNE3781-KR-XXXL    1     406
1       404-0687676-7273146  04-30-22    JNE3371-KR-XL    1     329
2       407-1069790-7240320  04-30-22  JNE3671-TU-XXXL    1     574
3       404-1490984-4578765  04-30-22  SET264-KR-NP-XL    1     824
4       408-5748499-6859555  04-30-22      J0095-SET-L    1     653
...                     ...       ...              ...  ...     ...
113696  406-6001380-7673107  05-31-22    JNE3697-KR-XL    1     517
113697  402-9551604-7544318  05-31-22   SET401-KR-NP-M    1     999
113698  407-9547469-3152358  05-31-22     J0157-DR-XXL    1     690
113699  402-6184140-0545956  05-31-22     J0012-SKD-XS    1    1199
113700  408-7436540-8728312  05-31-22      J0003-SET-S    1     696

[113701 rows x 5 columns]
Order ID    object
Date        object
SKU         object
Qty          int64
Amount       int64
dtype: object


- 1 month
- AMOUNT UNIDAD O CONJUNTO
- CHECK THE `SKU` & THE `AMOUNT`

In [5]:
# Agrupar por SKU y sumar la cantidad
result = df.groupby('SKU', as_index=False).agg(totalAmount=('Amount', 'sum')) #AGGREGATES COLUMN "TOTAL AMOUNT"

# Mostrar resultado
print(result)

                   SKU  totalAmount
0          AN201-RED-M          458
1         AN201-RED-XL          602
2        AN201-RED-XXL          229
3       AN202-ORANGE-M          530
4       AN202-ORANGE-S         1141
...                ...          ...
7108    SET449-KR-NP-S         1449
7109   SET461-KR-NP-XL         1039
7110   SET461-KR-NP-XS          769
7111  SET461-KR-NP-XXL          769
7112    SET462-KR-NP-S          999

[7113 rows x 2 columns]


In [6]:
# Ordenar por totalAmount
result = result.sort_values(by='totalAmount', ascending=False).reset_index(drop=True)

# Calcular acumulado in a new column
result['cumulativeSum'] = result['totalAmount'].cumsum()

# Mostrar resultados
print(result)

                   SKU  totalAmount  cumulativeSum
0          J0230-SKD-M       520526         520526
1         JNE3797-KR-L       480415        1000941
2          J0230-SKD-S       470605        1471546
3         JNE3797-KR-M       408332        1879878
4         JNE3797-KR-S       370852        2250730
...                ...          ...            ...
7108  JNE1975-KR-309-S          229       75401620
7109   AN208-MUSTARD-S          229       75401849
7110   AN209-BIEGE-XXL          229       75402078
7111   AN208-MUSTARD-M          229       75402307
7112   JNE3276-KR-XXXL          199       75402506

[7113 rows x 3 columns]


In [7]:
total_amount = result['totalAmount'].sum()
result['cumulativePercentage'] = result['totalAmount'].cumsum() / total_amount * 100

# Mostrar
print(result)

                   SKU  totalAmount  cumulativeSum  cumulativePercentage
0          J0230-SKD-M       520526         520526              0.690330
1         JNE3797-KR-L       480415        1000941              1.327464
2          J0230-SKD-S       470605        1471546              1.951588
3         JNE3797-KR-M       408332        1879878              2.493124
4         JNE3797-KR-S       370852        2250730              2.984954
...                ...          ...            ...                   ...
7108  JNE1975-KR-309-S          229       75401620             99.998825
7109   AN208-MUSTARD-S          229       75401849             99.999129
7110   AN209-BIEGE-XXL          229       75402078             99.999432
7111   AN208-MUSTARD-M          229       75402307             99.999736
7112   JNE3276-KR-XXXL          199       75402506            100.000000

[7113 rows x 4 columns]


It calculates the total amount for each `SKU`, computes the cumulative sum of these amounts, and then derives the cumulative percentage of each SKU relative to the overall total. This is useful for understanding the distribution of amounts across different SKUs and identifying which SKUs contribute most to the total.

In [10]:
fig = px.line(result, x=result.index, y='cumulativePercentage', markers=True,
              title='Cumulative Percentage by SKU Order',
              labels={'x': 'Order', 'cumulativePercentage': 'Cumulative Percentage (%)'})

fig.data[0].line.color = 'purple'

fig.show()


```
fig = px.line(result, x='order', y='cumulativePercentage', markers=True,
              title='Cumulative Percentage by SKU Order', labels={'order': 'Order', 'cumulativePercentage': 'Cumulative Percentage (%)'},
              hover_data={'order': False, 'SKU': True, 'cumulativePercentage': True})  # Show SKU in tooltip

fig.show()

ValueError                                Traceback (most recent call last)
<ipython-input-13-ca3e92d043cd> in <cell line: 0>()
----> 1 fig = px.line(result, x=result.index, y='cumulativePercentage', markers=True,
      2               title='Cumulative Percentage by SKU Order', labels={'order': 'Order', 'cumulativePercentage': 'Cumulative Percentage (%)'},
      3               hover_data={'order': False, 'SKU': True, 'cumulativePercentage': True})  # Show SKU in tooltip
      4
      5 fig.show()

3 frames
/usr/local/lib/python3.11/dist-packages/plotly/express/_core.py in process_args_into_dataframe(args, wide_mode, var_name, value_name)
   1232                         if argument == "index":
   1233                             err_msg += "\n To use the index, pass it in directly as `df.index`."
-> 1234                         raise ValueError(err_msg)
   1235                 elif length and len(df_input[argument]) != length:
   1236                     raise ValueError(

ValueError: Value of 'hover_data_0' is not the name of a column in 'data_frame'. Expected one of ['SKU', 'totalAmount', 'cumulativeSum', 'cumulativePercentage'] but received: order
```



In [14]:
fig = px.line(result, x=result.index, y='cumulativePercentage', markers=True,
              title='Cumulative Percentage by SKU Order', labels={'order': 'Order', 'cumulativePercentage': 'Cumulative Percentage (%)'},
              hover_data={'SKU': True, 'cumulativePercentage': True})  # Show SKU in tooltip

fig.show()


In [15]:
# Definir ABC
result['ABC'] = result['cumulativePercentage'].apply(lambda x: 'A' if x <= 60 else ('B' if x <= 80 else 'C'))

# Display
print(result)

                   SKU  totalAmount  cumulativeSum  cumulativePercentage ABC
0          J0230-SKD-M       520526         520526              0.690330   A
1         JNE3797-KR-L       480415        1000941              1.327464   A
2          J0230-SKD-S       470605        1471546              1.951588   A
3         JNE3797-KR-M       408332        1879878              2.493124   A
4         JNE3797-KR-S       370852        2250730              2.984954   A
...                ...          ...            ...                   ...  ..
7108  JNE1975-KR-309-S          229       75401620             99.998825   C
7109   AN208-MUSTARD-S          229       75401849             99.999129   C
7110   AN209-BIEGE-XXL          229       75402078             99.999432   C
7111   AN208-MUSTARD-M          229       75402307             99.999736   C
7112   JNE3276-KR-XXXL          199       75402506            100.000000   C

[7113 rows x 5 columns]


It classifies `SKU`s into three categories (A, B, C) based on their cumulative percentage of total amounts. This classification can be useful for inventory management, prioritizing stock, or analyzing product performance. The output shows the updated DataFrame with the new ABC classification, allowing for easy identification of which SKUs fall into each category.

In [16]:
#vamos a imprimir el valor de un SKU concreto
print(result[result['SKU'] == 'JNE3797-KR-S']['ABC'].iloc[0])

A


In [17]:
print(len(result[result['ABC'] == 'A']))
print(len(result[result['ABC'] == 'B']))
print(len(result[result['ABC'] == 'C']))

893
1076
5144
